In [3]:
from PIL import Image
from readlif.reader import LifFile
import numpy as np
import napari
import ipywidgets as widgets
import os
from skimage import filters, util

In [4]:
def collect_images(raw_data, img_sel):
    image_dict_list = []

    # check how many stacks, channels and mosaics there are
    raw_image = raw_data.get_image(img_sel)
    image_name = raw_image.name
    scale = tuple(abs(1/x) for x in raw_image.scale if x != None)
    z_nr_list = list(range(0,len([i for i in raw_image.get_iter_z()])))
    c_nr_list = list(range(0,len([i for i in raw_image.get_iter_c()])))
    m_nr_list = list(range(0,len([i for i in raw_image.get_iter_m()])))
    DimX = raw_image.dims[0]
    DimY = raw_image.dims[1]
    DimZ = raw_image.dims[2]

    # get info on if it is a mosaic tile
    M_positions = raw_image.mosaic_position # list with lenght of nr mosaics, each tuple contains (FieldX, FieldY, PosX, PosY)
    mosaic_list = list(range(0,len(M_positions)))
    mosaic_indication = 1
    if mosaic_list == []:
        mosaic_list = [0]
        mosaic_indication = 0 

    for m in m_nr_list:
        # collect info
        image_dict = {}
        image_dict['image_name'] = image_name
        image_dict['i_nr'] = img_sel
        image_dict['m_nr'] = m
        image_dict['scale'] = scale
        image_dict['M_positions'] = M_positions
        image_dict['DimX'] = DimX
        image_dict['DimY'] = DimY
        image_dict['DimZ'] = DimZ
        image_dict['bitdepth'] = raw_image.info['bit_depth'][0]
        # create 2d numpy array's per channel
        channel_dict = {}
        for c_nr in c_nr_list:
            for z_nr in z_nr_list:
                if z_nr == 0:
                    layers = []
                
                layer = np.asarray(raw_image.get_frame(z = z_nr, t = 0, c = c_nr, m = m))
                layers.append(layer)
            layers = np.stack(layers, axis = 2)  
            image_dict[f'channel_{c_nr}_image'] = layers
        
        image_dict_list.append(image_dict)
    return image_dict_list

def get_rgb(color):
    if color == 'blue':
        rgb = (0,0,1)
    elif color == 'green':
        rgb = (0,1,0)
    elif color == 'red':
        rgb = (1,0,0)
    elif color == 'yellow':
        rgb = (1,1,0)
    elif color == 'cyan':
        rgb = (0,1,1)
    elif color == 'magenta':
        rgb = (1,0,1)
    return rgb

def convert_intensity_range(img, target_type_min, target_type_max, target_type, min_quantile = False, max_quantile = False, input_bits = False):
    imin = img.min()
    # if no input bits is given, we want to transform the intensity range, not the type
    if input_bits == False:
        # correct if the image is binary and find min value
        if imin == False:
            img = img*1
            imin = img.min()
        if min_quantile != False:
            imin = np.quantile(img, q = min_quantile)
            
        # find max value
        imax = img.max()
        if max_quantile != False:
            imax = np.quantile(img, q = max_quantile)
    # if the input bits is given, we want to change the bitdepth, so we want to use the entire range
    else:
        if input_bits == 8:
            imin = 0
            imax = 255
        elif input_bits == 16:
            imin = 0
            imax = 65535
        elif input_bits == 32:
            imin = 0
            imax = 2147483647
    
    # build new image
    a = (target_type_max - target_type_min) / (imax - imin)
    b = target_type_max - a * imax
    new_img = (a * img + b)
    new_img[new_img>255] = 255
    new_img[new_img<0] = 0
    new_img = new_img.astype(target_type)
    return new_img

In [5]:
location = 'test-files'
file_list = [f for f in os.listdir(location) if os.path.isfile(os.path.join(location, f))]

file_selection = widgets.Dropdown(options = file_list, description = 'Image name')
display(file_selection)

Dropdown(description='Image name', options=('SA_22.083_08-07-2022-new.lif', '22.088_22.103_TH_20220816.lif', '…

In [6]:
raw_data = LifFile(f"{location}/{file_selection.value}")
image_list = list(range(0,len([i for i in raw_data.get_iter_image()])))
image_name_list = []
for i in image_list:
    raw_image = raw_data.get_image(i)
    image_name_list.append(raw_image.name)

image_selection = widgets.Dropdown(options = ['All'] + image_name_list, description = 'Image name')
display(image_selection)

Dropdown(description='Image name', options=('All', 'Plate1/A2', 'Plate1/A3', 'Plate1/A4', 'Plate1/A1', 'Plate1…

In [7]:
# select the right order of colors in the image: (channel 0 has color 0, channel 1 has color 1 etc...)
color_list = ['blue', 'green', 'red', 'yellow', 'magenta', 'cyan', 'gray']

# select the folder in which you want to save your images
maximum_projection_folder = 'maximum_projection_images'

# load the data and get a list of all image numbers
raw_data = LifFile(f"{location}/{file_selection.value}")

if image_selection.value == 'All':
    image_list = list(range(0,len([i for i in raw_data.get_iter_image()])))
else:
    image_list = [n for n, x in enumerate(image_name_list) if x == image_selection.value]

# make the maximum projection images per image
for i in image_list:
    # get the raw image
    raw_image = raw_data.get_image(i)
    # image_name = raw_image.name.split('/')[-1]
    image_name = raw_image.name.replace('/', '-')
    image_dict_list = collect_images(raw_data, i)
    for image_dict in image_dict_list:
        max_dict = {}
        for n, (name, channel_image) in enumerate([(name, img) for name, img in image_dict.items() if 'channel' in name ]):
            # make 8 bit
            max_channel_image = convert_intensity_range(channel_image, 0, 255, input_bits=image_dict['bitdepth'], target_type=np.uint8)
            # save the maximum projection image of this channel, and assign a color
            image_dict[f'{name}_max_projection'] = {"image" : max_channel_image, 'color' : color_list[n]}


    # create a dictionary in which we can store all our maximum projection images
    # this will only contain one 'frame' if there are no mosaic tiles, but will have a big image when mosaic tiles are present
    max_projection_image_dict = {}
    for image_dict in image_dict_list:
        # find image dimensions and check for presence of mosaic tiles
        DimX = image_dict['DimX']
        DimY = image_dict['DimY']
        MaxZ = image_dict['DimZ']
        M_positions = image_dict['M_positions']

        # if mosaic tiles exist, build up the framework for the large image
        if M_positions != []:
            # if it is the first image that passes by, create the large image template
            if len(max_projection_image_dict.keys())==0:
                channel_image_list = [x for x in image_dict.keys() if 'max_projection' in x]

                # find MaxX and MaxY to build up the empty image dimensions
                MaxX = max(M_positions, key= lambda x: x[0])[0]
                MaxY = max(M_positions, key= lambda x: x[1])[1]
                empty_image = np.empty(((DimY * (MaxY+1)), (DimX * (MaxX+1)), MaxZ))
                
                # loop over the images and assign a color for them as well
                for channel_image_name in channel_image_list:
                    image, color = image_dict[channel_image_name].values()
                    max_projection_image_dict[channel_image_name] = {}
                    max_projection_image_dict[channel_image_name]['image'] = empty_image.copy()
                    max_projection_image_dict[channel_image_name]['color'] = color

            # find sub-image specific x-y coords        
            m_nr = image_dict['m_nr']
            FieldX = M_positions[m_nr][0]
            FieldY = M_positions[m_nr][1]

            # find coordinates in merged image
            Min_PixX = FieldX * DimX
            Max_PixX = ((FieldX+1) * DimX)
            Min_PixY = FieldY * DimY
            Max_PixY = ((FieldY+1) * DimY)

            # fill the large images with the sub_images
            for channel_image_name in [x for x in image_dict.keys() if 'max_projection' in x]:
                # max_projection_image_dict[channel_image_name]['image'][Min_PixY:Max_PixY, Min_PixX:Max_PixX] = image_dict[channel_image_name]['image']
                max_projection_image_dict[channel_image_name]['image'][Min_PixY:Max_PixY, Min_PixX:Max_PixX, 0:MaxZ] = image_dict[channel_image_name]['image']

        # if there are no mosaics, just save the maximum projection images
        else:
            # loop over channels and build up the max_projection_image_dict
            for channel_image_name in [x for x in image_dict.keys() if 'max_projection' in x]:
                image, color = image_dict[channel_image_name].values()
                max_projection_image_dict[channel_image_name] = {}
                max_projection_image_dict[channel_image_name]['image'] = image
                max_projection_image_dict[channel_image_name]['color'] = color

    # build maximum projection image
    empty_image = np.zeros_like(max_projection_image_dict['channel_0_image_max_projection']['image'])
    default_color_image = np.dstack((empty_image, empty_image, empty_image))
    red_image = default_color_image[:,:,0]
    green_image = default_color_image[:,:,1]
    blue_image = default_color_image[:,:,2]

    # build the masked image, start with the default image
    for channel_info in max_projection_image_dict:
        # get the image and the color
        image, color = max_projection_image_dict[channel_info].values()
        image = np.max(image, axis = 2)
        # change the color into rgb signals
        rgb = get_rgb(color)
        # fill in the image in the right color channels
        if rgb[0] > 0:
            red_image = np.dstack((red_image, image))
        if rgb[1] > 0:
            green_image = np.dstack((green_image, image))
        if rgb[2] > 0:
            blue_image = np.dstack((blue_image, image))
    
    red_image = np.max(red_image, axis = 2)
    green_image = np.max(green_image, axis = 2)
    blue_image = np.max(blue_image, axis = 2)
    
    # red_image = convert_intensity_range(red_image, 0, 255, min_quantile = 0.1, max_quantile = 0.98, target_type = np.uint8)
    # green_image = convert_intensity_range(green_image, 0, 255, min_quantile = 0.1, max_quantile = 0.98, target_type = np.uint8)
    # blue_image = convert_intensity_range(blue_image, 0, 255, min_quantile = 0.1, max_quantile = 0.98, target_type = np.uint8)
    
    # create the color image with dimensions (x, y, (r, g, b))
    color_image = np.dstack((red_image, green_image, blue_image))
    image_for_saving = Image.fromarray(convert_intensity_range(color_image, 0, 255, np.uint8, input_bits=8))   
    if image_selection.value == 'All':
        #save the image with the correct name in the correct location
        image_for_saving.save(f"{maximum_projection_folder}/{image_name}.png")
    else:
        try:
            viewer.close()
        except:
            print('No viewer was opened yet.')
        viewer = napari.Viewer()
        viewer.add_image(color_image)